In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as preprocessing 
from sklearn.decomposition import PCA
import tensorflow as tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from sklearn.metrics import confusion_matrix

# Load MNIST Dataset

In [162]:
# Training Data
Train = pd.read_csv('/home/tonzhu/Downloads/mnist_train.csv', header = None)
print(Train.shape)
Train.head(10)

(60000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
Train_data = Train.iloc[:,1:]
Train_label_or = Train.iloc[:,0]

In [164]:
# Test Data
Test = pd.read_csv('/home/tonzhu/Downloads/mnist_test.csv', header = None)
print(Test.shape)
Test.head(10)

(10000, 785)


,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
Test_data = Test.iloc[:,1:]
Test_label_or = Test.iloc[:,0]

# Data Preprocessing

In [60]:
# Normalization (min_max, make number between 0 and 1)
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True).fit(Train_data)
X_train_minmax = minmax_scale.transform(Train_data)
X_test_minmax = minmax_scale.transform(Test_data)

In [102]:
# Copy training data for shuffle (Original Data)
X_df = Train_data.copy()
X_df['label'] = Train_label
X_df2 = X_df.copy()

In [ ]:
# Copy training data for shuffle (Normalized Data)
X_copy = X_train_minmax.copy()
X_df = pd.DataFrame(X_copy)
X_df['label'] = Train_label
X_df2 = X_df.copy()

In [103]:
# Training 
Train_data_or = Train_data.copy().values
Test_data_or = Test_data.copy().values

In [84]:
# Training 
Train_data_or = X_train_minmax.copy()
Test_data_or = X_test_minmax.copy()

In [63]:
# Create One-hot encoding for labels
Train_label_oh_or = pd.get_dummies(Train_label).values
Test_label_oh_or = pd.get_dummies(Test_label).values

In [64]:
tf.set_random_seed(1)
np.random.seed(1)

# CNN Training

In [254]:
# Construct CNN architecture
tf.reset_default_graph()

# Constants
training_epochs = 100
batch_size = 500
total_batch = int(len(Train) / batch_size)
learning_rate = 0.001 
display_step = 10

# Variables
x = tf.placeholder(tf.float32, [None, 28*28])
y = tf.placeholder(tf.int32, [None, 10])            # input y
image = tf.reshape(x, [-1, 28, 28, 1])              # (batch, height, width, channel)

# CNN
conv1 = tf.layers.conv2d(   # shape (28, 28, 1)
    inputs = image,
    filters = 8,
    kernel_size = 5,
    strides = 1,
    padding = 'same',
    activation = tf.nn.relu
) # (28, 28, 32)
pool1 = tf.layers.max_pooling2d(
    conv1,
    pool_size=2,
    strides=2,
) # (14, 14, 32)
conv2 = tf.layers.conv2d(pool1, 16, 5, 1, 'same', activation=tf.nn.relu)  # (14, 14, 64)
pool2 = tf.layers.max_pooling2d(conv2, 2, 2)    # (7, 7, 64)
conv3 = tf.layers.conv2d(pool2, 32, 5, 1, 'same', activation=tf.nn.relu)  # (14, 14, 64)
flat = tf.reshape(conv3, [-1, 7*7*32])          # (7*7*64, )
dp1 = tf.layers.dropout(flat, rate = 0.5)
fc1 = tf.layers.dense(dp1, 120)
dp2 = tf.layers.dropout(fc1, rate = 0.5)
fc2 = tf.layers.dense(dp2, 84)
dp3 = tf.layers.dropout(fc2, rate = 0.5)
output = tf.layers.dense(dp3, 10)              # output
decode_label = tf.argmax(output, axis = 1)

# Loss function, optimizer and results
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=output) 
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
accuracy = tf.metrics.accuracy(labels=tf.argmax(y, axis=1), predictions=tf.argmax(output, axis=1),)[1] # get accuracy

# Initializing
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

# Training and Test
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    step = 1
    for epoch in xrange(training_epochs):
        # Transfer training data to 28 x 28 size
        Train_data = X_df2.iloc[:,:-1].values
        Train_label = X_df2.iloc[:,-1]
        Train_label_oh = pd.get_dummies(Train_label).values
        for i in xrange(total_batch):
            batch_x = Train_data[i * batch_size : (i + 1) * batch_size,:]
            batch_y = Train_label_oh[i * batch_size : (i + 1) * batch_size,:]
            sess.run(optimizer, feed_dict = {x: batch_x, y: batch_y})
            if step % display_step == 0:
                acc = sess.run(accuracy, feed_dict = {x: batch_x, y: batch_y})
                cost = sess.run(loss, feed_dict = {x: batch_x, y: batch_y})
                print  ('Iter ' + str(step * batch_size) + ': Minibatch Loss = ' +
                       '{: .6f}'.format(cost) + ', Training Accuracy = ' +
                       '{: .5f}'.format(acc))
            step += 1
        # Shuffle Data
        X_df2 = X_df2.sample(frac = 1).reset_index(drop = True)
    print ('Optimization Finished!')
    # Training, Test Results
    train_predict = np.array([])
    for i in xrange(total_batch):
        part_x = Train_data_or[i * batch_size : (i + 1) * batch_size]
        train_predict = np.append(train_predict, sess.run(decode_label, {x: part_x}))
    test_predict = np.array([])
    for i in xrange(total_batch):
        part_x_test = Test_data_or[i * batch_size : (i + 1) * batch_size]
        test_predict = np.append(test_predict, sess.run(decode_label, {x: part_x_test}))
            

Iter 5000: Minibatch Loss =  5.488602, Training Accuracy =  0.39400
Iter 10000: Minibatch Loss =  1.252140, Training Accuracy =  0.49100
Iter 15000: Minibatch Loss =  0.784186, Training Accuracy =  0.57733
Iter 20000: Minibatch Loss =  0.492920, Training Accuracy =  0.64850
Iter 25000: Minibatch Loss =  0.427106, Training Accuracy =  0.69120
Iter 30000: Minibatch Loss =  0.316214, Training Accuracy =  0.72533
Iter 35000: Minibatch Loss =  0.254157, Training Accuracy =  0.75257
Iter 40000: Minibatch Loss =  0.256042, Training Accuracy =  0.77350
Iter 45000: Minibatch Loss =  0.272972, Training Accuracy =  0.78911
Iter 50000: Minibatch Loss =  0.193736, Training Accuracy =  0.80340
Iter 55000: Minibatch Loss =  0.231710, Training Accuracy =  0.81400
Iter 60000: Minibatch Loss =  0.162159, Training Accuracy =  0.82633
Iter 65000: Minibatch Loss =  0.160541, Training Accuracy =  0.83615
Iter 70000: Minibatch Loss =  0.147597, Training Accuracy =  0.84486
Iter 75000: Minibatch Loss =  0.153

Iter 595000: Minibatch Loss =  0.005709, Training Accuracy =  0.96936
Iter 600000: Minibatch Loss =  0.021293, Training Accuracy =  0.96953
Iter 605000: Minibatch Loss =  0.013251, Training Accuracy =  0.96975
Iter 610000: Minibatch Loss =  0.021848, Training Accuracy =  0.96995
Iter 615000: Minibatch Loss =  0.013664, Training Accuracy =  0.97016
Iter 620000: Minibatch Loss =  0.002049, Training Accuracy =  0.97040
Iter 625000: Minibatch Loss =  0.016172, Training Accuracy =  0.97059
Iter 630000: Minibatch Loss =  0.006462, Training Accuracy =  0.97081
Iter 635000: Minibatch Loss =  0.020204, Training Accuracy =  0.97101
Iter 640000: Minibatch Loss =  0.008008, Training Accuracy =  0.97122
Iter 645000: Minibatch Loss =  0.012946, Training Accuracy =  0.97143
Iter 650000: Minibatch Loss =  0.043950, Training Accuracy =  0.97157
Iter 655000: Minibatch Loss =  0.012983, Training Accuracy =  0.97177
Iter 660000: Minibatch Loss =  0.011642, Training Accuracy =  0.97194
Iter 665000: Minibat

Iter 1180000: Minibatch Loss =  0.004566, Training Accuracy =  0.98375
Iter 1185000: Minibatch Loss =  0.002838, Training Accuracy =  0.98381
Iter 1190000: Minibatch Loss =  0.008711, Training Accuracy =  0.98386
Iter 1195000: Minibatch Loss =  0.017375, Training Accuracy =  0.98390
Iter 1200000: Minibatch Loss =  0.013548, Training Accuracy =  0.98394
Iter 1205000: Minibatch Loss =  0.007690, Training Accuracy =  0.98399
Iter 1210000: Minibatch Loss =  0.008037, Training Accuracy =  0.98404
Iter 1215000: Minibatch Loss =  0.011705, Training Accuracy =  0.98410
Iter 1220000: Minibatch Loss =  0.002398, Training Accuracy =  0.98416
Iter 1225000: Minibatch Loss =  0.005008, Training Accuracy =  0.98422
Iter 1230000: Minibatch Loss =  0.006735, Training Accuracy =  0.98427
Iter 1235000: Minibatch Loss =  0.008834, Training Accuracy =  0.98432
Iter 1240000: Minibatch Loss =  0.003389, Training Accuracy =  0.98439
Iter 1245000: Minibatch Loss =  0.000387, Training Accuracy =  0.98445
Iter 1

Iter 1760000: Minibatch Loss =  0.001546, Training Accuracy =  0.98861
Iter 1765000: Minibatch Loss =  0.002420, Training Accuracy =  0.98863
Iter 1770000: Minibatch Loss =  0.010422, Training Accuracy =  0.98866
Iter 1775000: Minibatch Loss =  0.001104, Training Accuracy =  0.98869
Iter 1780000: Minibatch Loss =  0.000716, Training Accuracy =  0.98872
Iter 1785000: Minibatch Loss =  0.003257, Training Accuracy =  0.98875
Iter 1790000: Minibatch Loss =  0.008980, Training Accuracy =  0.98877
Iter 1795000: Minibatch Loss =  0.002741, Training Accuracy =  0.98880
Iter 1800000: Minibatch Loss =  0.004933, Training Accuracy =  0.98882
Iter 1805000: Minibatch Loss =  0.001902, Training Accuracy =  0.98885
Iter 1810000: Minibatch Loss =  0.004722, Training Accuracy =  0.98888
Iter 1815000: Minibatch Loss =  0.009540, Training Accuracy =  0.98890
Iter 1820000: Minibatch Loss =  0.000825, Training Accuracy =  0.98893
Iter 1825000: Minibatch Loss =  0.001431, Training Accuracy =  0.98896
Iter 1

Iter 2340000: Minibatch Loss =  0.000905, Training Accuracy =  0.99121
Iter 2345000: Minibatch Loss =  0.002819, Training Accuracy =  0.99123
Iter 2350000: Minibatch Loss =  0.001596, Training Accuracy =  0.99125
Iter 2355000: Minibatch Loss =  0.008084, Training Accuracy =  0.99127
Iter 2360000: Minibatch Loss =  0.006220, Training Accuracy =  0.99128
Iter 2365000: Minibatch Loss =  0.007513, Training Accuracy =  0.99129
Iter 2370000: Minibatch Loss =  0.001361, Training Accuracy =  0.99131
Iter 2375000: Minibatch Loss =  0.000764, Training Accuracy =  0.99133
Iter 2380000: Minibatch Loss =  0.001377, Training Accuracy =  0.99135
Iter 2385000: Minibatch Loss =  0.001406, Training Accuracy =  0.99137
Iter 2390000: Minibatch Loss =  0.003147, Training Accuracy =  0.99138
Iter 2395000: Minibatch Loss =  0.000952, Training Accuracy =  0.99140
Iter 2400000: Minibatch Loss =  0.001171, Training Accuracy =  0.99142
Iter 2405000: Minibatch Loss =  0.015278, Training Accuracy =  0.99143
Iter 2

...

Iter 3500000: Minibatch Loss =  0.003147, Training Accuracy =  0.99387
Iter 3505000: Minibatch Loss =  0.001483, Training Accuracy =  0.99388
Iter 3510000: Minibatch Loss =  0.006201, Training Accuracy =  0.99389
Iter 3515000: Minibatch Loss =  0.003225, Training Accuracy =  0.99389
Iter 3520000: Minibatch Loss =  0.004123, Training Accuracy =  0.99390
Iter 3525000: Minibatch Loss =  0.000355, Training Accuracy =  0.99391
Iter 3530000: Minibatch Loss =  0.003619, Training Accuracy =  0.99392
Iter 3535000: Minibatch Loss =  0.000946, Training Accuracy =  0.99392
Iter 3540000: Minibatch Loss =  0.020646, Training Accuracy =  0.99393
Iter 3545000: Minibatch Loss =  0.001045, Training Accuracy =  0.99394
Iter 3550000: Minibatch Loss =  0.000398, Training Accuracy =  0.99394
Iter 3555000: Minibatch Loss =  0.002611, Training Accuracy =  0.99395
Iter 3560000: Minibatch Loss =  0.001255, Training Accuracy =  0.99396
Iter 3565000: Minibatch Loss =  0.000345, Training Accuracy =  0.99397
Iter 3

Iter 4080000: Minibatch Loss =  0.000396, Training Accuracy =  0.99466
Iter 4085000: Minibatch Loss =  0.000510, Training Accuracy =  0.99467
Iter 4090000: Minibatch Loss =  0.000376, Training Accuracy =  0.99467
Iter 4095000: Minibatch Loss =  0.006519, Training Accuracy =  0.99468
Iter 4100000: Minibatch Loss =  0.001515, Training Accuracy =  0.99468
Iter 4105000: Minibatch Loss =  0.000207, Training Accuracy =  0.99469
Iter 4110000: Minibatch Loss =  0.000207, Training Accuracy =  0.99470
Iter 4115000: Minibatch Loss =  0.000276, Training Accuracy =  0.99470
Iter 4120000: Minibatch Loss =  0.000558, Training Accuracy =  0.99471
Iter 4125000: Minibatch Loss =  0.000134, Training Accuracy =  0.99472
Iter 4130000: Minibatch Loss =  0.000451, Training Accuracy =  0.99472
Iter 4135000: Minibatch Loss =  0.000104, Training Accuracy =  0.99473
Iter 4140000: Minibatch Loss =  0.000546, Training Accuracy =  0.99473
Iter 4145000: Minibatch Loss =  0.000435, Training Accuracy =  0.99474
Iter 4

Iter 4660000: Minibatch Loss =  0.000069, Training Accuracy =  0.99526
Iter 4665000: Minibatch Loss =  0.002594, Training Accuracy =  0.99526
Iter 4670000: Minibatch Loss =  0.002472, Training Accuracy =  0.99527
Iter 4675000: Minibatch Loss =  0.003602, Training Accuracy =  0.99527
Iter 4680000: Minibatch Loss =  0.002486, Training Accuracy =  0.99527
Iter 4685000: Minibatch Loss =  0.000618, Training Accuracy =  0.99528
Iter 4690000: Minibatch Loss =  0.000299, Training Accuracy =  0.99528
Iter 4695000: Minibatch Loss =  0.000622, Training Accuracy =  0.99529
Iter 4700000: Minibatch Loss =  0.001303, Training Accuracy =  0.99529
Iter 4705000: Minibatch Loss =  0.000320, Training Accuracy =  0.99530
Iter 4710000: Minibatch Loss =  0.000402, Training Accuracy =  0.99530
Iter 4715000: Minibatch Loss =  0.000466, Training Accuracy =  0.99531
Iter 4720000: Minibatch Loss =  0.000982, Training Accuracy =  0.99531
Iter 4725000: Minibatch Loss =  0.001523, Training Accuracy =  0.99532
Iter 4

Iter 5240000: Minibatch Loss =  0.000496, Training Accuracy =  0.99574
Iter 5245000: Minibatch Loss =  0.000237, Training Accuracy =  0.99574
Iter 5250000: Minibatch Loss =  0.000926, Training Accuracy =  0.99575
Iter 5255000: Minibatch Loss =  0.001366, Training Accuracy =  0.99575
Iter 5260000: Minibatch Loss =  0.011158, Training Accuracy =  0.99575
Iter 5265000: Minibatch Loss =  0.001985, Training Accuracy =  0.99576
Iter 5270000: Minibatch Loss =  0.000009, Training Accuracy =  0.99576
Iter 5275000: Minibatch Loss =  0.000315, Training Accuracy =  0.99576
Iter 5280000: Minibatch Loss =  0.000283, Training Accuracy =  0.99577
Iter 5285000: Minibatch Loss =  0.000119, Training Accuracy =  0.99577
Iter 5290000: Minibatch Loss =  0.000575, Training Accuracy =  0.99578
Iter 5295000: Minibatch Loss =  0.000578, Training Accuracy =  0.99578
Iter 5300000: Minibatch Loss =  0.000166, Training Accuracy =  0.99578
Iter 5305000: Minibatch Loss =  0.007406, Training Accuracy =  0.99579
Iter 5

Iter 5820000: Minibatch Loss =  0.000023, Training Accuracy =  0.99613
Iter 5825000: Minibatch Loss =  0.000202, Training Accuracy =  0.99613
Iter 5830000: Minibatch Loss =  0.002066, Training Accuracy =  0.99613
Iter 5835000: Minibatch Loss =  0.000151, Training Accuracy =  0.99613
Iter 5840000: Minibatch Loss =  0.002055, Training Accuracy =  0.99614
Iter 5845000: Minibatch Loss =  0.000489, Training Accuracy =  0.99614
Iter 5850000: Minibatch Loss =  0.000588, Training Accuracy =  0.99614
Iter 5855000: Minibatch Loss =  0.000030, Training Accuracy =  0.99615
Iter 5860000: Minibatch Loss =  0.009062, Training Accuracy =  0.99615
Iter 5865000: Minibatch Loss =  0.000692, Training Accuracy =  0.99615
Iter 5870000: Minibatch Loss =  0.000064, Training Accuracy =  0.99615
Iter 5875000: Minibatch Loss =  0.000240, Training Accuracy =  0.99616
Iter 5880000: Minibatch Loss =  0.000284, Training Accuracy =  0.99616
Iter 5885000: Minibatch Loss =  0.000357, Training Accuracy =  0.99616
Iter 5

In [255]:
accuracy_CNN = float(np.trace(confusion_matrix(Train_label_or.values, train_predict))) / float(len(Train))

In [256]:
accuracy_CNN

0.9993666666666666

In [257]:
accuracy_CNN_test = float(np.trace(confusion_matrix(Test_label_or.values, test_predict))) / float(len(Test))

In [258]:
accuracy_CNN_test

0.9876